In [1]:
Web_page_search_context = "novartis Iptacopan competitors"
number_of_pages = 10
chunk_size = 1000
chunk_overlap =20
docs_to_vectorize = 2
embeddings_to_use ='Ollama'
model_to_use = 'Ollama'

from pathlib import Path
import os
import yaml
from pathlib import Path 
from box.exceptions import BoxValueError


CUSTOM_ENV_PATH = lambda a : '/'.join(os.get_exec_path()[0].split('\\')[:-2])+'/'+a
CUSTOM_ENV_NAME = CUSTOM_ENV_PATH('chatter.env')

def extract_links ( ac, link_in_dict, dict_key ='link' ):
    """
    reads a list of dictionaries and returns specific value for a key 
    """
    link_in_list = [ link_in_dict[i][dict_key] for i in range(len(link_in_dict)) ]
    print ( "Activity ", ac, ": Done: extracting webpage from dict")
    return link_in_list


In [2]:
import os
from pathlib import Path
from dotenv import load_dotenv
load_dotenv(CUSTOM_ENV_NAME)

os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")
os.environ["GOOGLE_CSE_ID"] = os.getenv("GOOGLE_CSE_ID")

from langchain_community.utilities import GoogleSearchAPIWrapper


def extract_related_links ():
    """
    read context and number of results from params.yaml
    returns a text file with saved list of web pages to be vectorized
    """
    # extract the context and length of search for websites
    search_context =Web_page_search_context
    search_num = number_of_pages
    print (search_context, "for ", search_num, " pages")

    # use the context and length to do a google search using an api wrapper
    api_wrapper = GoogleSearchAPIWrapper( siterestrict = False )
    link_in_dict = api_wrapper.results (search_context  , search_num )
    print ( "Activity 2: Done: getting webpages from Google wrapper")
    # Extract the links from the above dictionary to a list so as to be used for webbasedloader
    link_in_list =  extract_links( 3,  link_in_dict , 'link')

    return link_in_list

list_link = extract_related_links()

novartis Iptacopan competitors for  10  pages


d:\Study2\git\Environments\novas\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `GoogleSearchAPIWrapper` was deprecated in LangChain 0.0.33 and will be removed in 0.3.0. An updated version of the class exists in the langchain-google-community package and should be used instead. To use it run `pip install -U langchain-google-community` and import as `from langchain_google_community import GoogleSearchAPIWrapper`.
  warn_deprecated(


Activity 2: Done: getting webpages from Google wrapper
Activity  3 : Done: extracting webpage from dict


In [3]:
list_link

['https://www.novartis.com/news/media-releases/novartis-investigational-iptacopan-phase-iii-study-demonstrates-clinically-meaningful-and-highly-statistically-significant-proteinuria-reduction-patients-iga-nephropathy-igan',
 'https://www.biospace.com/article/novartis-pnh-trial-results-put-pressure-on-astrazeneca-s-ultomiris-and-soliris/',
 'https://www.novartis.com/research-development/novartis-pipeline',
 'https://www.biopharmadive.com/news/novartis-iptacopan-pnh-alexion-complement/634793/',
 'https://pharmaphorum.com/news/novartis-oral-drug-iptacopan-tops-injectables-in-head-to-head-pnh-trial',
 'https://seekingalpha.com/article/4566810-novartis-competitors-take-note-iptacopan-is-coming',
 'https://www.fiercebiotech.com/biotech/novartis-eyeing-3b-market-beats-astrazeneca-blockbuster-phase-3-trigger-race-regulators',
 'https://www.clinicaltrialsarena.com/news/novartis-iptacopan-pnh/',
 'https://www.fiercebiotech.com/biotech/novartis-rolls-out-encouraging-phase-2-data-rare-kidney-disea

In [4]:
import yaml
from pathlib import Path 
import os

from src.constants import *
from src.utils import read_yaml, read_from_text

from dotenv import load_dotenv
load_dotenv(CUSTOM_ENV_NAME)

from langchain_community.document_loaders import WebBaseLoader
import bs4
from langchain_text_splitters import RecursiveCharacterTextSplitter


from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.embeddings import HuggingFaceInstructEmbeddings
from InstructorEmbedding import INSTRUCTOR
from langchain_community.vectorstores import FAISS

from langchain_core.prompts import ChatPromptTemplate
from langchain import hub
from langchain_community.chat_models import ChatOllama
from langchain_community.llms import Ollama
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain

import streamlit as st
import time

def create_retreiver (links_to_load):
    """
    read chunk size and overlap from params.yaml
    """
    # # read the list of webpage to be taken
    # links_to_load = read_from_text ( 5, WEB_PAGE_TXT )

    # use a loader to get all content
    loader = WebBaseLoader(web_paths = links_to_load,)
                    #        bs_kwargs= dict(parse_only = bs4.SoupStrainer(
                    #        class_ = ("post-title","post-content","post-header")
                    #    )),)    
    web_docs = loader.load()
    print ("size of the web docs : ",len(web_docs))
    print ("Activity ", 6, ": Done: webpage content from links")
    
    # extract the size and overlap chars from param file
    print (chunk_size, "is the chunk size choosen with ", chunk_overlap, " chars overlaps ")

    # splitting the docs based on above size etc
    documents = RecursiveCharacterTextSplitter(chunk_size = chunk_size, chunk_overlap = chunk_overlap).split_documents(web_docs)
    # taking only top n docs
    documents_filtered = documents[:2]
    # choosing the embeddings to use
    if embeddings_to_use == "Instruct":
        print ( "Instruct Embeddings" )
        choosen_embeddings = HuggingFaceInstructEmbeddings(model_name = 'hkunlp/instructor-xl',  
                                                   model_kwargs={"device":"cpu"},
                                                   encode_kwargs ={'normalize_embeddings': True} )
    else :
        choosen_embeddings = OllamaEmbeddings()
        print ( "Ollama Embeddings" )
    # creatign vector store and eventually retreiver
    verctordb = FAISS.from_documents(documents_filtered, choosen_embeddings )
    #retriever = verctordb.as_retriever()
    return verctordb

def get_retrieval_chain (retriever):
    # get prompt
    # prompt = hub.pull("hwchase17/openai-functions-agent")
    # print( prompt.messages )
    prompt = ChatPromptTemplate.from_template(
    """
    Answer the questions based on the provided context only.
    Please provide the most accurate response based on the question
    <context>
    {context}
    <context>
    Questions: {input}
    """)

    #llm = Ollama(model = "llama2")
    llm= ChatOllama(model="llama2")

    # creating document chain
    document_chain = create_stuff_documents_chain(llm, prompt)
    retrieval_chain = create_retrieval_chain (retriever, document_chain)
    return retrieval_chain

def talk_to_stream (retrieval_chain):
    st.title("Demo with LLAMA2")

    input_text = st.text_input ("Input the question you have about Iptacopan ")
    #input_text = "what is Iptacopan"
    if input_text:
        start_time = time.process_time()
        response = retrieval_chain.invoke ({"input":input_text})
        print ("response time: ",time.process_time()-start_time)
        print ( response['answer'] )
        st.write(response['answer'])

d:\Study2\git\Environments\novas\Lib\site-packages\InstructorEmbedding\instructor.py:7: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import trange


In [5]:
# retr = create_retreiver (list_link)

size of the web docs :  10
Activity  6 : Done: webpage content from links
1000 is the chunk size choosen with  20  chars overlaps 
Ollama Embeddings


In [5]:
vcdb = create_retreiver (list_link)

size of the web docs :  10
Activity  6 : Done: webpage content from links
1000 is the chunk size choosen with  20  chars overlaps 
Ollama Embeddings


In [14]:
vcdb.save_local ( os.getcwd() , 'nova_vector_db')

In [20]:
os.getcwd()

'd:\\Study2\\git\\Leep_Ipta_QA_demo'

In [44]:
vcdb_load = FAISS.load_local ( 'd:/Study2/git/Leep_Ipta_QA_demo',index_name  = 'nova_vector_db',
                               embeddings = OllamaEmbeddings() ,allow_dangerous_deserialization = True   ) 

In [6]:
rc  = get_retrieval_chain (retr)

In [7]:
talk_to_stream ( rc )

2024-06-25 23:30:50.296 
  command:

    streamlit run d:\Study2\git\Environments\novas\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]


response time:  0.328125
Iptacopan is an investigational drug developed by Novartis for the treatment of IgA nephropathy (IgAN), a kidney disease. According to the context provided, the Phase III study of iptacopan has demonstrated clinically meaningful and highly statistically significant proteinuria reduction in patients with IgAN.
